In [1]:
# Access the telegram profiles and parses all context and responses in the profile.

In [2]:
from telethon import TelegramClient
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.types import User
import openai
from dotenv import load_dotenv
import json
import os
import sys

In [3]:
dotenv_path = ".env"
load_dotenv(dotenv_path=dotenv_path)

api_id = os.getenv('TELEGRAM_API_ID')
api_hash = os.getenv('TELEGRAM_HASH_ID')
session_name = "telegram_parser"
phone_number = os.getenv('PHONE_NUMBER')
client = TelegramClient(session_name, api_id, api_hash)


In [4]:
openai.api_key = os.getenv('OPENAI_API_KEY')

def categorize_message(message):
    prompt = f'Classify the following message as either "context" or "response":\n\n"{message}"\n\nAnswer with one word only.'
    
    response = openai.Completion.create(
        engine="text-davinci-003",  # Choose a suitable engine
        prompt=prompt,
        max_tokens=1,
        n=1,
        stop=None,
        temperature=0.5,
    )
    
    return response.choices[0].text.strip()

In [5]:
def optimize_messages(messages):
      """
      Function which uses a set of tuning algorithms to meet the criteria of optimized data for future models.
      """

      # TODO: Include only messages in ukrainian language


      # TODO: Put todos below in order of priority 
      # For each of the points below, if true: add one, if false: minus one
      # TODO: Add detection system for context and response:

      # TODO: If the message contains question mark in the end of the message, it is a context
      # TODO: The first message of the new day is probably a context.
      # TODO: If there are few messages in a row from user, concatenate them into one message.
      # TODO: If there is a significant time gap (e.g., several hours) between messages, the first message after the gap might be a context.
      # TODO: Look for specific keywords or phrases that typically indicate a context (e.g., "What do you think about...", "Can you explain...", "Why is...").
      # TODO: If a message is a direct reply to a previous context message, it is likely a response.
      # TODO: Short messages that directly follow a context are likely responses.
      # TODO: If the same user repeatedly sends messages ending with question marks or messages at the start of the day, those are likely contexts.
       

In [102]:
async def extract_message_info(messages):
      extracted_dialog = []
      last_message=None
      result = {}

      for message in messages:
            text = message.message
            sender = message.from_id if message.from_id else await client.get_entity(message.peer_id) 
            date = message.date

            if last_message and text:
                  if sender == last_message[1]:
                        last_message[0] = " ".join([last_message[0], text])
                  else:
                        extracted_dialog.append(last_message)
                        last_message = [text, sender, date]
            else:
                  last_message = [text, sender, date]

      if last_message:
            extracted_dialog.append(last_message)
      
      return extracted_dialog


In [101]:
async def parse_data(threshold=50):
    async with client:
        dialogs = await client.get_dialogs()
        dialogs = [dialog for dialog in dialogs if isinstance(dialog.entity, User)]
        dialogs = [dialog for dialog in dialogs if not dialog.entity.bot]
        filtered_dialogs = []
        for dialog in dialogs:
            messages = await client.get_messages(dialog.entity, limit=55)

            total_messages = len(messages)
            if total_messages > threshold:
                extracted_dialog = await extract_message_info(messages)
                return extracted_dialog
async def main():
    if os.path.exists(f"parsers\{session_name}.session-journal"):
        print(f"Session {session_name} exists. Please delete it and restart the script. Or change the session name in the script.")
        sys.exit()
    else:
        await client.start(phone_number)
        print(f"Connecting with {client.session}")
        data = await parse_data()
        client.disconnect()
        print("DONE")

await main()
#client.disconnect

Connecting with <telethon.sessions.sqlite.SQLiteSession object at 0x12627e490>


TypeError: list.append() takes exactly one argument (3 given)

In [ ]:
[Message(id=79743, peer_id=PeerUser(user_id=597029615), date=datetime.datetime(2024, 8, 13, 13, 20, 22, tzinfo=datetime.timezone.utc), message='', out=True, mentioned=False, media_unread=False, silent=False, post=False, from_scheduled=False, legacy=False, edit_hide=True, pinned=False, noforwards=False, invert_media=False, offline=False, from_id=PeerUser(user_id=1037155651), from_boosts_applied=None, saved_peer_id=None, fwd_from=None, via_bot_id=None, via_business_bot_id=None, reply_to=None, media=MessageMediaPhoto(spoiler=False, photo=Photo(id=5321321732667204545, access_hash=-8016869431638365570, file_reference=b'\x03\x00\x017\x7ff\xbc\xdc\rx\x99\xc1#\xc5\xbaamX\xd1\xee\xbc\xfe\xb6J\xd2', date=datetime.datetime(2024, 8, 13, 13, 20, 21, tzinfo=datetime.timezone.utc), sizes=[PhotoStrippedSize(type='i', bytes=b'\x01(\'\xd3\x9a\xe1atB\x8e\xcc\xf9\xc0Q\x9e\x94\xc3v\xbb\x95^)Sy\xc0,8\xfeu\x06\xa4\xbb\xa5\x8b9\nRE,\x14\x9cd\x0fJ\xabm\x18\x84E\x1a\xb39\xf3\xb7\x93\xe5\xb0\x00`\x8e\xe2\x804Zh\xd5\x15\xf2\xe5\x0e\xdc0<\x1c\x9aE\xbc\x0e7$\x130\xc9\x19\n?\xc6\xa0rL\x911\x91\x8c\x87\x07v\xdc\x05\x07\xb0\x1dy\xfd*\x94\xb0\t\xa1\x8d\x19\xda6F|\x8f)\x8fS\xec(\x03f\t\xd6p\xc5U\x97km!\x879\xa2\xa1\xd3\xf9Y\xdb\x04\x06\x94\x91\x90FF\x05\x14\x00\x97\xf2l\x8b\n\xd8\x99\x81X\xc6\xe23\xff\x00\xd7\xac\xef\xb4\\\xc4\xf2E5\xc7\xef\x86\x08\r\xc0\x1c\x0f\xcf\xe9W\xef\xe5h\xe4\x8b\x0e\xca\xbb]\x9bn2p\x01\xefU\xe1\xb93\x08\xe4G\x94\xa9\x97c+\xe3\x91\x8c\xfaP\x013\xca\xd0\x87o4N\xaa\xbb\xbeQ\xf2`\xf2\x7f\x1f\xe9V"\x92v\x0b\'\x06F\x1cF\xe7n\x17\xd7\x8f^?:\xab\xb5\xad\xe3\x1eL\xfb|\xc2\xc4F\x18q\xc0\xe3\'\x8e>\xb4\xf8\x18<V\xfecyO\x1b\x92\xcb\xb4\xe0\xf5\xe7\xff\x00\xaf\xd2\x80,io3\xdb\xb8\x9d\x89tr\xbc\xf5\x1c\n)l\x14\x84\x98\x92X4\xa5\x81=H\xc0\xebE\x00>\xea\xd4\xdc22\xc9\xb0\xa0#\xee\x86\xc8?Z\x85,\x19YI\x98\x15V\xdd\xb5b\x0b\x93\xd3\xb5\x14P\x04o\xa5\x06#l\xbbF9\x1b22z\x9ct\x15j\x08\x08B\xb38\x98ch\xdc\x80`zQE\x00:\xd6\x03o\x19V}\xe4\x9c\xee\xc6\x0f\xff\x00^\x8a(\xa0\x0f'), PhotoSize(type='m', w=310, h=320, size=17002), PhotoSize(type='x', w=774, h=800, size=42357), PhotoSizeProgressive(type='y', w=1239, h=1280, sizes=[10199, 20198, 30674, 40215, 59682])], dc_id=2, has_stickers=False, video_sizes=[]), ttl_seconds=None), reply_markup=None, entities=[], views=None, forwards=None, replies=None, edit_date=datetime.datetime(2024, 8, 13, 13, 21, 14, tzinfo=datetime.timezone.utc), post_author=None, grouped_id=None, reactions=MessageReactions(results=[ReactionCount(reaction=ReactionEmoji(emoticon='🥰'), count=1, chosen_order=None)], min=False, can_see_list=False, reactions_as_tags=False, recent_reactions=[MessagePeerReaction(peer_id=PeerUser(user_id=597029615), date=datetime.datetime(2024, 8, 13, 13, 21, 14, tzinfo=datetime.timezone.utc), reaction=ReactionEmoji(emoticon='🥰'), big=False, unread=False, my=False)]), restriction_reason=[], ttl_period=None, quick_reply_shortcut_id=None, effect=None, factcheck=None),

In [95]:
main_array = [1,2,3,4]

main_array.append([1, 2,3,4])

In [96]:
main_array

[1, 2, 3, 4, [1, 2, 3, 4]]

In [ ]:
client = TelegramClient('telegram_parse', api_id, api_hash)
dialogs = client.get_dialogs()
print(dialogs[:2])
client.disconnect()